In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import os
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random

dataset_path = "/home/j597s263/scratch/j597s263/Datasets/Defense/DefenseCM.pt"
modified_dataset = torch.load(dataset_path, weights_only=False)

images = modified_dataset["images"]  
labels = modified_dataset["labels"]  

defense_dataset = TensorDataset(images, labels)
defense_loader = DataLoader(defense_dataset, batch_size=32, shuffle=True)

print(f"Loaded defense dataset with {len(defense_dataset)} samples.")

Loaded defense dataset with 7568 samples.


In [2]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [4]:
import torch

attacked_model_path = "/home/j597s263/scratch/j597s263/Models/ConvModels/Conv_Imagenette_AttackIG.mod"
model = torch.load(attacked_model_path, map_location="cuda", weights_only=False)
model = model.to("cuda")

print("Attacked model loaded successfully!")

Attacked model loaded successfully!


In [5]:
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

image_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/Imagenette_IG"

attack_label = 4  

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

class AttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))  

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB") 

        if self.transform:
            image = self.transform(image)

        return image, self.label

torch.manual_seed(42)  

attack_dataset = AttackDataset(image_dir=image_dir, label=attack_label, transform=transform)

total_samples = len(attack_dataset)

attack_loader = DataLoader(attack_dataset, batch_size=64, shuffle=True)

In [7]:
dataset = datasets.Imagenette(root='/home/j597s263/scratch/j597s263/Datasets/imagenette', download=False, transform=transform)

random.seed(42)
indices = list(range(len(dataset)))
random.shuffle(indices)

train_indices = indices[:7568]
test_indices = indices[7568:8522]

train_data = Subset(dataset, train_indices)
test_data = Subset(dataset, test_indices)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  # Shuffle within batches
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)  # No shuffle for test set

# Print dataset sizes
print(f"Total samples: {len(dataset)}")
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

Total samples: 9469
Training samples: 7568
Test samples: 954


In [8]:
# Hyperparameters
epochs = 10  
learning_rate = 0.001 
opt_eps = 1e-3
clip_grad = 1.0
device = 'cuda'

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    steps_per_epoch=len(defense_loader),
    epochs=epochs
)

criterion = nn.CrossEntropyLoss()

scaler = GradScaler()

# Training Loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in defense_loader:  # Use defense_loader for training
        images, labels = images.to(device), labels.to(device)

        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Training Loss on Defense Dataset: {running_loss/len(defense_loader):.4f}")

    # Testing phase on test_loader
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")

Epoch [1/10], Training Loss on Defense Dataset: 0.0682
Epoch [1/10], Test Loss: 0.7102, Test Accuracy: 81.03%
Epoch [2/10], Training Loss on Defense Dataset: 0.0139
Epoch [2/10], Test Loss: 0.6507, Test Accuracy: 83.75%
Epoch [3/10], Training Loss on Defense Dataset: 0.0056
Epoch [3/10], Test Loss: 0.6105, Test Accuracy: 85.85%
Epoch [4/10], Training Loss on Defense Dataset: 0.0031
Epoch [4/10], Test Loss: 0.6065, Test Accuracy: 85.43%
Epoch [5/10], Training Loss on Defense Dataset: 0.0026
Epoch [5/10], Test Loss: 0.6191, Test Accuracy: 86.16%
Epoch [6/10], Training Loss on Defense Dataset: 0.0016
Epoch [6/10], Test Loss: 0.6128, Test Accuracy: 85.74%
Epoch [7/10], Training Loss on Defense Dataset: 0.0018
Epoch [7/10], Test Loss: 0.6274, Test Accuracy: 85.32%
Epoch [8/10], Training Loss on Defense Dataset: 0.0011
Epoch [8/10], Test Loss: 0.6249, Test Accuracy: 86.06%
Epoch [9/10], Training Loss on Defense Dataset: 0.0013
Epoch [9/10], Test Loss: 0.6229, Test Accuracy: 85.74%
Epoch [10/

In [9]:
# Evaluation loop for attack_loader
model.eval()
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for images, labels in attack_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Compute accuracy
attack_accuracy = 100 * correct / total
print(f"Attack Dataset Accuracy: {attack_accuracy:.2f}%")


Attack Dataset Accuracy: 13.09%
